In [ ]:
import pandas as pd
import numpy as np
import torch

if torch.cuda.is_available():
    # Move your tensors and model to the GPU
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU")

GPU is available


In [ ]:
import pandas as pd
import numpy as np
import torch as torch
import re
from os import listdir
import requests
import transformers
from transformers import XLNetTokenizer, AutoModel, pipeline, AutoTokenizer, T5EncoderModel, T5Tokenizer
from tqdm.auto import tqdm
import gc
from IPython.display import display

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.set_num_threads (4)

#function to read in the data
def read_in (file):
    data = pd.read_csv (file, engine='python')
    return data

def new_features(features):
    ids = tokenizer.batch_encode_plus(features, add_special_tokens=True, padding=True)
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)
    with torch.no_grad():
        embedding = model(input_ids=input_ids,attention_mask=attention_mask)
    embedding = embedding.last_hidden_state.cpu().numpy()
    new_feature = []
    for seq_num in range(len(embedding)):
        seq_len = (attention_mask[seq_num]==1).sum()
        seq_emd = embedding[seq_num][:seq_len-1]
        new_feature.append(seq_emd)
    out = torch.tensor (new_feature)
    out = out.mean(1)
    out = out.reshape(-1)
    out = out.tolist()

    return out

#loads the T5_xl_uniref50 model for the generation of the embeddings
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
gc.collect()


filepath_base = 'tcr_epitope_interactions.csv'
filepath = filepath_base
data = read_in (filepath)
data = data.dropna(subset=['CDR3B'])
data = data.dropna(subset=['Epitope'])
data['CDR3B'] = data['CDR3B'].apply(eval)
data['Epitope'] = data['Epitope'].apply(eval)

#generate embeddings for each protein
try:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model = model.eval()
    data['CDR3B'] = data['CDR3B'].apply (lambda x: new_features (x))
    data['Epitope'] = data['Epitope'].apply (lambda x: new_features (x))
except:
    device = torch.device ('cpu')
    model = model.to(device)
    model = model.eval()
    data['CDR3B'] = data['CDR3B'].apply (lambda x: new_features (x))
    data['Epitope'] = data['Epitope'].apply (lambda x: new_features (x))

#safe the adjusted dataframe
safepath = 'tcr_epitope_interactions_embedded.csv'
data.to_csv (safepath, index=False)
print ('done')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

<ipython-input-1-834220b84297>:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  out = torch.tensor (new_feature)


done


In [ ]:
!pip install pyarrow

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Read CSV file into a DataFrame
df = pd.read_csv('testing_dataset.csv')

# Convert DataFrame to PyArrow Table
table = pa.Table.from_pandas(df)

# Write PyArrow Table to Parquet file
pq.write_table(table, 'testing_dataset.parquet')


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data"

mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab Notebooks/data’: File exists


In [ ]:
! cp testing_dataset.parquet "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data"

tcr_epitope_interactions_embedded.parquet  testing_dataset.parquet
